In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import json

In [8]:
import src.music.mir
reload(src.music.mir)
from src.music.mir import MusicInformationRetreiver

mir = MusicInformationRetreiver(weights_path='mir_weights/')

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mir_weights//discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mir_weights//mtg_jamendo_genre-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mir_weights//mtg_jamendo_instrument-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mir_weights//msd-musicnn-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mir_weights//emomusic-msd-musicnn-2.pb`


In [17]:
def plot_segments(segments):
    segments.sort(key=lambda x: x['start'])
    fig, ax = plt.subplots(1, 1, figsize=(15, 5), sharex=True)

    colors = plt.cm.rainbow(np.linspace(0, 1, len(segments)))
    for idx, seg in enumerate(segments):
        rect = plt.Rectangle(
            xy=(seg['start'], 0), 
            width=seg['end'] - seg['start'], 
            height=1, facecolor=colors[idx], alpha=0.5, edgecolor='black'
        )
        ax.add_patch(rect)
    ax.set_title('Segments')
    ax.set_ylim(0, 1)
    ax.set_yticks([])
    ax.set_xlabel('Time')
    ax.set_xlim(min(seg['start'] for seg in segments), max(seg['end'] for seg in segments))
    plt.tight_layout()
    return fig

def plot_emotions(segments):
    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    valence = []
    arousal = []
    ax.set_title('Emotion Dimenions')
    for seg in segments:
        valence += [np.mean(seg['valence'])]
        arousal += [np.mean(seg['arousal'])]
    ax.plot(valence, label='valence')
    ax.plot(arousal, label='arousal')
    plt.legend()
    return fig


def music_fn(music_path, segment_threshold, instrument_threshold, genre_threshold):
    segments = mir(
        music_path, 
        segment_threshold=segment_threshold, 
        genre_threshold=genre_threshold, 
        inst_threshold=instrument_threshold, 
        add_segment_info=True,
        progress_bar=gr.Progress().tqdm
    )
    seg_plot = plot_segments(segments)
    emo_plot = plot_emotions(segments)
    for seg in segments:
        del seg['arousal']
        del seg['valence']
    return seg_plot, emo_plot, json.dumps(segments, indent=4)

music_demo = gr.Interface(
    fn=music_fn,
    inputs=[
        gr.Audio(label="Upload Music File", type='filepath', sources='upload'),
        gr.Slider(minimum=0.01, step=0.01, maximum=0.99, value=0.1, label="Segmentation Threshold"),
        gr.Slider(minimum=0.01, step=0.01, maximum=0.99, value=0.3, label="Instrumentation Threshold"),
        gr.Slider(minimum=0.01, step=0.01, maximum=0.99, value=0.3, label="Genre Threshold"),
    ],
    outputs=[
        gr.Plot(label="Music Segments", format="png"),
        gr.Plot(label="Segment Emotion", format="png"),
        gr.Text(label='Segments Details')
    ]
)
music_demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7870
Running on public URL: https://27fa892b90ed017928.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
